In [249]:
%load_ext autoreload
%autoreload 2
import sys
print(sys.executable)
import socket
print(socket.gethostname())
import os
os.chdir("/home/icb/alioguz.can/projects/he_pathalogy")

import numpy as np
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/icb/alioguz.can/miniconda3/envs/he_env/bin/python
supergpu14.scidom.de


In [297]:
import pydicom
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tifffile as tiff
import squidpy as sq
import seaborn  as sns
import histomicstk as htk
from collections import defaultdict
from pathlib import Path
from tqdm import tqdm
from utils import find_full_tiles, plot_top_tiles, quick_tissue_mask, macenko_stain_matrix, separate_stains_od

In [273]:
path = "/lustre/groups/ml01/workspace/alioguz.can/hne_cg/filtered_dicoms"

# Collect all DICOMs
all_files = sorted([p for p in os.listdir(path) if p.endswith(".dcm")])
print(f"🔍 Scanning {len(all_files)} DICOM files in: {path}\n")

# Group by prefix (before first underscore)
groups = defaultdict(list)
for f in all_files:
    prefix = f.split("_")[0]
    groups[prefix].append(f)

# Find duplicate groups
duplicates = {k: v for k, v in groups.items() if len(v) > 1}

if duplicates:
    print("⚠️ Duplicate slide prefixes found:\n")
    for prefix, files in duplicates.items():
        print(f"[{prefix}] — {len(files)} DICOMs")
        for f in files:
            full_path = os.path.join(path, f)
            try:
                ds = pydicom.dcmread(full_path, stop_before_pixels=True)
                rows, cols = ds.get("Rows", "?"), ds.get("Columns", "?")
                frames = ds.get("NumberOfFrames", "N/A")
                print(f"   {f} → {rows}×{cols} pixels, {frames} frames")
            except Exception as e:
                print(f"   {f} → [ERROR reading DICOM: {e}]")
        print()
else:
    print("✅ No duplicates by prefix found.\n")

# Summary
print("—" * 80)
print(f"📦 Total DICOM files scanned: {len(all_files)}")
print(f"📂 Duplicate prefix groups:   {len(duplicates)}")
print("—" * 80)

🔍 Scanning 48 DICOM files in: /lustre/groups/ml01/workspace/alioguz.can/hne_cg/filtered_dicoms

✅ No duplicates by prefix found.

————————————————————————————————————————————————————————————————————————————————
📦 Total DICOM files scanned: 48
📂 Duplicate prefix groups:   0
————————————————————————————————————————————————————————————————————————————————


In [ ]:
### DELTES DUPLICATE FILES OF SHARED PARENT FOLDERS BY KEEPING THE FILE WITH MORE FRAMES

# path = "/lustre/groups/ml01/workspace/alioguz.can/hne_cg/filtered_dicoms"

# # Collect all DICOMs
# all_files = sorted([p for p in os.listdir(path) if p.endswith(".dcm")])
# print(f"🔍 Scanning {len(all_files)} DICOM files in: {path}\n")

# # Group by prefix (before first underscore)
# groups = defaultdict(list)
# for f in all_files:
#     prefix = f.split("_")[0]
#     groups[prefix].append(f)

# # Find duplicate groups
# duplicates = {k: v for k, v in groups.items() if len(v) > 1}

# deleted_count = 0
# kept_count = 0

# if duplicates:
#     print("⚠️ Duplicate slide prefixes found and processing for cleanup:\n")
#     for prefix, files in duplicates.items():
#         print(f"[{prefix}] — {len(files)} DICOMs")

#         frame_info = []
#         for f in files:
#             full_path = os.path.join(path, f)
#             try:
#                 ds = pydicom.dcmread(full_path, stop_before_pixels=True)
#                 frames = int(ds.get("NumberOfFrames", 1))
#                 frame_info.append((frames, full_path))
#                 print(f"   {f} → {frames} frames")
#             except Exception as e:
#                 print(f"   {f} → [ERROR reading DICOM: {e}]")

#         # Sort by frame count (highest last)
#         frame_info.sort(key=lambda x: x[0])
#         keep = frame_info[-1]  # highest frame count
#         to_delete = frame_info[:-1]

#         kept_count += 1
#         print(f"   ✅ Keeping: {os.path.basename(keep[1])} ({keep[0]} frames)")

#         # Delete lower-frame duplicates
#         for frames, path_to_remove in to_delete:
#             try:
#                 os.remove(path_to_remove)
#                 deleted_count += 1
#                 print(f"   ❌ Deleted: {os.path.basename(path_to_remove)} ({frames} frames)")
#             except Exception as e:
#                 print(f"   [ERROR deleting {path_to_remove}: {e}]")

#         print()
# else:
#     print("✅ No duplicates by prefix found.\n")

# # Summary
# print("—" * 80)
# print(f"📦 Total DICOM files scanned:  {len(all_files)}")
# print(f"📂 Duplicate prefix groups:    {len(duplicates)}")
# print(f"✅ Files kept (highest frames): {kept_count}")
# print(f"❌ Files deleted (lower frames): {deleted_count}")
# print("—" * 80)

In [280]:
root_path = Path("/lustre/groups/ml01/workspace/alioguz.can/hne_cg/filtered_dicoms")

# Get all .dcm files
all_files = sorted([p for p in os.listdir(root_path) if p.endswith(".dcm")])

# Extract prefixes (before the first underscore)
prefixes = sorted({f.split("_")[0] for f in all_files})

# Print results
print(f"📦 Total DICOM files: {len(all_files)}")
print(f"🔹 Unique prefixes: {len(prefixes)}\n")

for p in prefixes:
    print(p)

📦 Total DICOM files: 50
🔹 Unique prefixes: 50

E20012878#1#3#3
E20012878#1#4#3
E20012878#1#5#3
E21004292#2#1#3
E21006287#1#1#3
E21006287#3#1#3
E21006287#4#1#3
E21006287#5#1#3
E21006287#6#1#3
E21006288#1#1#3
E21006288#2#1#3
E21006288#3#1#3
E21006288#4#1#3
E21006288#5#1#3
E21006679#1#1#3
E21006679#2#1#3
E21006679#3#1#3
E21009820#1#1#5
E21011531#10#1#3
E21011531#11#1#3
E21011531#2#1#3
E21011531#3#1#3
E21011531#5#1#3
E21011531#6#1#3
E21011531#7#1#3
E21011531#8#1#3
E21011531#9#1#3
KO21000706#1#1#3
KO21000706#10#1#3
KO21000706#11#1#3
KO21000706#2#1#3
KO21000706#4#1#3
KO21000706#6#1#3
KO21000706#7#1#3
KO21000739#2#1#3
KO21001954#2#1#3
KO21001972#2#1#3
KO21001972#3#1#3
KO21001972#4#1#3
KO21001972#7#1#3
KO21001972#8#1#3
KO21001994#1#1#3
KO21001994#1#2#3
KO21001994#2#1#3
KO21001994#2#2#3
KO21001996#1#1#3
KO21001997#1#1#3
KO21001997#3#1#3
KO21001997#5#1#3
KO21001997#7#1#3


In [295]:
dcm_file_paths = sorted(root_path.rglob("*.dcm"))
print(len(dcm_file_paths))

50


In [ ]:
patient_dict = {}

for dicom_path in tqdm(dcm_file_paths, desc="Processing paths"):
    print(f"\nProcessing {dicom_path.name} ...")

    ds = pydicom.dcmread(dicom_path)
    image = ds.pixel_array.astype(np.float32)

    # Handle possible planar configuration
    if getattr(ds, "PlanarConfiguration", 0) == 1:
        image = np.moveaxis(image, 0, -1)  # (3, H, W) → (H, W, 3)
    elif image.shape[-1] != 3:
        image = np.transpose(image, (1, 2, 0))  # (3, H, W) → (H, W, 3)

    # Ensure (N, H, W, 3)
    if image.ndim == 3:
        image = image[None, ...]

    # --- Per-tile normalization ---
    p_low, p_high = 1, 99.5
    for i in range(image.shape[0]):
        tile = image[i]
        lo, hi = np.percentile(tile, [p_low, p_high])
        tile = np.clip((tile - lo) / (hi - lo + 1e-6), 0, 1)
        image[i] = tile * 255.0
    image = image.astype(np.uint8)

    # print("normalized shape:", image.shape, "dtype:", image.dtype)

    # Patient ID
    patient_id = ds.ContainerIdentifier
    # print(f"Patient: {patient_id}")

    # Skip if already processed
    if patient_id not in patient_dict:
        patient_dict[patient_id] = {}

    # --- Find full tiles ---
    full_idx, scores = find_full_tiles(image, SCALE_255=False)
    # print(f"{len(full_idx)} files have signal.")
    full_idx = full_idx

    # --- Compute Eosin OD for full tiles ---
    E_od_list = []
    for idx in full_idx:
        tile = image[idx]
        tmask = quick_tissue_mask(tile)
        W = macenko_stain_matrix(tile, tissue_mask=tmask, beta=0.15, alpha=1.0)
        H_od, E_od, _ = separate_stains_od(tile, W)
        E_od_list.append(E_od)
    E_od_array = np.stack(E_od_list)

    # --- Segmentation and counting ---
    for i, arr in enumerate(E_od_array):
        sq_imgCont = sq.im.ImageContainer(arr, layer="image", mask=None, dims="channels_last")
        sq.im.process(sq_imgCont, layer="image", method="smooth", sigma=1)
        threshold_val = np.percentile(np.array(sq_imgCont["image_smooth"]).flatten(), 99)
        sq.im.segment(img=sq_imgCont, layer="image_smooth", method="watershed", thresh=threshold_val, geq=True)

        seg_mask = sq_imgCont["segmented_watershed"].values
        labels = np.unique(seg_mask)
        n_objects = len(labels) - (1 if 0 in labels else 0)

        print(f"Tile {full_idx[i]}: Thresholding at {threshold_val} -> Number of objects identified: {n_objects}")
        patient_dict[patient_id][f"tile_{full_idx[i]}"] = n_objects

# --- Convert dictionary to DataFrame ---
df = pd.DataFrame.from_dict(patient_dict, orient="index").fillna(0).astype(int)
df = df.reindex(sorted(df.columns, key=lambda x: int(x.split("_")[1])), axis=1)
df["maximum"] = df.apply(
    lambda row: f"{row.idxmax()}={row.max()}", axis=1
)
df.to_csv("/home/icb/alioguz.can/projects/he_pathalogy/hne_objects_by_patient_and_tile.csv")
print(f"\nPatients: {df.shape[0]} | Tiles: {df.shape[1]}")